### Quantum Cellular Automata based on Conway's Game of Life

In [ ]:
import pygame
import numpy as np
import os
from sys import exit
from IPython.display import display


# Qiskit imports
from qiskit import BasicAer, QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit.circuit.library import RXXGate as XX
from qiskit.circuit.library import RZZGate as ZZ

from qiskit.aqua.operators import X, Y, Z, I, CX, T, H, S, PrimitiveOp
from qiskit.aqua.operators import StateFn, Zero, One, Plus, Minus, H, DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn
from qiskit.aqua.operators import I, X, Y, Z, H, CX, Zero, ListOp, \
                                  PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki
from qiskit.circuit import Parameter

# Cirq and Tensorflow imports
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import sympy
from cirq.contrib.svg import SVGCircuit

In [ ]:
class Cell(object):
    simulation_framework = ' '
    num_layers = 1
    # Cirq version
    strength_circuit = None
    symbols = []
    operators = None
    strength_expectation = tfq.layers.Expectation()
    
    # Qiskit version
    unitary_params = []
    rule_circuit = []
    unitary_blocks = []
    
    def __init__(self, x, y, birthplace, size):
        self.old_state = np.array([0, 0, 0])
        self.new_state = np.array([0, 0, 0])
        self.stored_state = np.array([0, 0, 0])
        self.x = x
        self.y = y
        self.birthplace = birthplace
        self.size = size
        self.neighbours = []
        
    
    def get_form(self):
        return pygame.Rect(self.x, self.y, self.size, self.size)
    
    def detect_neighbours(self, universe):
        x, y = self.birthplace
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                if i == 0 and j == 0:
                    continue
                key = (x + (self.size + 1) * i, y + (self.size + 1) * j)
                try:
                    if key in universe.keys() and key not in [self.neighbours[i].birthplace for i in range(len(self.neighbours))]:
                        self.neighbours.append(universe.get(key))
                except:
                    continue
    
        
   
    def construct_unitary_block(layer):
        # general rotations of Pauli X, Y and Z
        from qiskit.aqua.operators import X, Y, Z
        circuit = I
        p = Parameter('theta_Y' + str(layer))
        Cell.unitary_params.append(p)
        circuit = (p * Y).exp_i() @ circuit

        p = Parameter('theta_Z' + str(layer))
        Cell.unitary_params.append(p)
        circuit = (p * Z).exp_i() @ circuit

        p = Parameter('theta_X' + str(layer))
        Cell.unitary_params.append(p)
        circuit = (p * X).exp_i() @ circuit
                
        return circuit     
      
    
    
    def construct_quantum_rules_qiskit():
        rule_circuit = Zero
        for layer in range(Cell.num_layers):
            #if i == 0:
            Cell.unitary_blocks.append(Cell.construct_unitary_block(layer))
            rule_circuit = Cell.unitary_blocks[-1] @ rule_circuit
        Cell.rule_circuit = rule_circuit
    
    
    

    def construct_quantum_rules_cirq():
        moore_nbhd = [cirq.GridQubit(-1, i) for i in range(8)]
        cell = cirq.GridQubit(0, 0)
        Cell.strength_circuit = cirq.Circuit()
        Cell.symbols = []
            
        for j in range(Cell.num_layers):
            for k in range(3):
                rot = sympy.Symbol('p_' + str(k) + str(j))
                Cell.symbols.append(rot)
                if k == 0:
                    Cell.strength_circuit.append(cirq.Y(cell) ** (rot))
                elif k == 1:
                    Cell.strength_circuit.append(cirq.Z(cell) ** (rot))
                else:
                    Cell.strength_circuit.append(cirq.X(cell) ** (rot))
        
        Cell.operators = [cirq.Y(cell), cirq.Z(cell), cirq.X(cell)]
        
            
        
    def compute_state(self, option):
        if option == 'vanilla':
            # Conway's version of game of life
            num_alive = 0
            for neighbour in self.neighbours:
                num_alive += int(np.round(neighbour.old_state[0]))
            if num_alive < 2 or num_alive > 3:
                self.new_state = np.array([0, 0, 0])
            if num_alive == 2:
                self.new_state = np.array([int(np.round(self.old_state[0])), 0, 0])
            if num_alive == 3:
                self.new_state = np.array([1, 0, 0])
        
        elif option == 'continuous':
            # Continuous version of game of life 
            alive_strength = 0.0
            for neighbour in self.neighbours:
                alive_strength += neighbour.old_state[0]
            if self.old_state[0] <= 0.2:
                if alive_strength <= 2:
                    self.new_state = np.array([0, 0, 0])
                elif alive_strength > 2 and alive_strength <= 3:
                    self.new_state = np.array([(alive_strength - 2), 0, 0])
                elif alive_strength > 3 and alive_strength <= 4:
                    self.new_state = np.array([(4 - alive_strength), 0, 0])
                else:
                    self.new_state = np.array([0, 0, 0])
            else:
                if alive_strength <= 1:
                    self.new_state = np.array([0, 0, 0])
                elif alive_strength > 1 and alive_strength <= 2:
                    self.new_state = np.array([(alive_strength - 1), 0, 0])
                elif alive_strength > 2 and alive_strength <= 3:
                    self.new_state = np.array([1, 0, 0])
                elif alive_strength > 3 and alive_strength <= 4:
                    self.new_state = np.array([(4 - alive_strength), 0, 0])
                else:
                    self.new_state = np.array([0, 0, 0])
    
            
            
    def evolve_and_measure(self):
        alive_strengths = []
        calc_cell_strength = np.array([])
        for k in range(3):
            count = 0
            for neighbour in self.neighbours:
                count += 1
                alive_strengths.append(neighbour.old_state[k])
            while count < 8:
                count += 1
                alive_strengths.append(0.0)
                
        if sum(alive_strengths) == 0:
            self.new_state = np.array([0] * 3)
            return
        
        if Cell.simulation_framework == 'Cirq':
            calc_cell_strength = Cell.strength_expectation(Cell.strength_circuit,
                                                          symbol_names = Cell.symbols,
                                                          symbol_values = [np.pi/2 * (2 * np.array([sum(alive_strengths[:8]), sum(alive_strengths[8:16]), sum(alive_strengths[16:24])]) - 8)], # 1.595 (diagonal growth), 3.06 (exponentially growing squares, has period of 4), np.pi/2 (oscillator)
                                                          operators = Cell.operators)
            calc_cell_strength = np.clip((1 - np.real(calc_cell_strength.numpy()[0])) / 2, 0, 1)
        
        elif Cell.simulation_framework == 'Qiskit':
            resolved_circuit = Cell.rule_circuit.bind_parameters(dict(zip(Cell.unitary_params, np.pi/2 * np.pi/2 * (2 * np.array([sum(alive_strengths[:8]), sum(alive_strengths[8:16]), sum(alive_strengths[16:24])]) - 8))))
            calc_cell_strength = [np.real(PauliExpectation().convert(observable @ resolved_circuit).eval()) for observable in [~StateFn(Y), ~StateFn(Z), ~StateFn(X)]]
            calc_cell_strength = np.clip((1 - np.array(calc_cell_strength)) / 2, 0, 1)
            
        alive_strength = calc_cell_strength
        self.new_state = []
        for i in range(3):
            if self.old_state[i] <= 0.5:
                if alive_strength[i] <= 0.2:
                    self.new_state.append(0.0)
                elif alive_strength[i] > 0.2 and alive_strength[i] <= 0.3:
                    self.new_state.append(1.0)
                else:
                    self.new_state.append(0.0)
            else:
                if alive_strength[i] <= 0.1:
                    self.new_state.append(0.0)
                elif alive_strength[i] > 0.2 and alive_strength[i] <= 4:
                    self.new_state.append(1.0)
                else:
                    self.new_state.append(0.0)
        self.new_state = np.array(self.new_state)

        



class GameOfLife(object):
    def __init__(self, simulation_framework = 'Classical', game_of_life_version = 'vanilla'):
        """
        Pass appropriate arguments for choosing the simulation type.
        Please read the README.md file on github to know more about
        the simulation types.
        Vanilla version : simulation_framework='Classical'
                          game_of_life_version='vanilla'
                          
        Continuous version : simulation_framework='Classical'
                             game_ofLife_version='continuous'
                             
        Quantum version (Qiskit) : simulation_framework='Qiskit'
        
        Quantum version (Cirq) : simulation_framework='Cirq'
        
        """
        Cell.simulation_framework = simulation_framework
        self.game_of_life_version = game_of_life_version
        self.SCREEN_WIDTH = 1020
        self.SCREEN_HEIGHT = 720
        self.WINDOW = None
        self.FPS = 50
        self.curr_color = 0
        self.block_size = 20
        self.camera_x = 0
        self.camera_y = 0
        self.Cells = {}
        self.captured_images = 0
        self.prevCells = None
        self.start_life = False
        self.change_dir = False
        self.main()
        
    def init_game(self):
        self.WINDOW = pygame.display.set_mode((self.SCREEN_WIDTH, self.SCREEN_HEIGHT)) 
        pygame.display.set_caption("Game of Life")
        if Cell.simulation_framework == 'Qiskit':
            Cell.construct_quantum_rules_qiskit()
        elif Cell.simulation_framework == 'Cirq':
            Cell.construct_quantum_rules_cirq()
            
    
    def detect_cell(self, mycell):
        for cell in self.Cells:
            if cell.get_form().colliderect(mycell.get_form()):
                return True
        return False
    
    
    def draw_grid(self):
        dir_x = int(np.sign(self.camera_x + 0.0001))
        dir_y = int(np.sign(self.camera_y + 0.0001))
        start_x = (1 - int((1 + dir_x)/2)) * (self.SCREEN_WIDTH)
        end_x = dir_x * (self.SCREEN_WIDTH) + self.camera_x
        start_y = (1 - int((1 + dir_y)/2)) * (self.SCREEN_HEIGHT)
        end_y = dir_y * (self.SCREEN_HEIGHT) + self.camera_y
        step_x = dir_x * self.block_size
        step_y = dir_y * self.block_size
        
        for x in range(start_x, end_x, step_x):
            for y in range(start_y, end_y, step_y):
                if (x, y) not in self.Cells.keys():
                    cell = Cell(x - self.camera_x, y - self.camera_y, (x, y), self.block_size - 1)
                    self.Cells.update({(x, y) : cell})
        for cell in self.Cells.values():
            cell.detect_neighbours(self.Cells)
        self.render()

                
    def render(self):
        self.WINDOW.fill((255, 255, 255))
        for i, cell in enumerate(self.Cells.values()):
            pygame.draw.rect(self.WINDOW, tuple((255 * cell.old_state).astype(int)), cell.get_form())
        
        # Take screenshots
        #if self.start_life:
        #   pygame.image.save(self.WINDOW, "screenshot_{}.jpg".format(self.captured_images))
        #   self.captured_images += 1
        

    
    
    def move_camera(self, keys_pressed):
        distance_moved_x = 0
        distance_moved_y = 0
        if keys_pressed[pygame.K_w]: # UP
            self.camera_y -= 5
            distance_moved_y += 5
        if keys_pressed[pygame.K_a]: # LEFT
            self.camera_x -= 5
            distance_moved_x += 5
        if keys_pressed[pygame.K_s]: # DOWN
            self.camera_y += 5
            distance_moved_y -= 5
        if keys_pressed[pygame.K_d]: # RIGHT
            self.camera_x += 5
            distance_moved_x -= 5
        
        for cell in self.Cells.values():
            cell.x += distance_moved_x
            cell.y += distance_moved_y
        
        self.draw_grid()
        self.render()
          
    
    def evolve_universe(self):
        for cell in self.Cells.values():
            if Cell.simulation_framework == 'Qiskit' or Cell.simulation_framework == 'Cirq':
                # hybrid classical quantum automata
                cell.evolve_and_measure()
            elif Cell.simulation_framework == 'Classical':
                # Continuous or simple Game of life
                cell.compute_state(self.game_of_life_version)
           
        # Updating cell states
        for cell in self.Cells.values():
            cell.old_state = cell.new_state
            
        self.render()
    
    
    
    def select_cell(self, pos):
        for cell in self.Cells.values():
            if cell.get_form().collidepoint(pos):
                if not self.change_dir:
                    cell.old_state = cell.old_state + np.array([0.1, 0, 0])
                else:
                    cell.old_state = cell.old_state - np.array([0.1, 0, 0])
                
                if cell.old_state[0] >= 1:
                    self.change_dir = True
                    cell.old_state[0] = 1
                
                if cell.old_state[0] <= 0:
                    self.change_dir = False
                    cell.old_state[0] = 0
        self.render()
        
        
    def store_grid(self):
        for cell in self.Cells.values():
            cell.stored_state = cell.old_state

            
            
    def restore_grid(self):
        for cell in self.Cells.values():
            cell.old_state = cell.stored_state
        

        
    def main(self):
        self.init_game()
        self.draw_grid()
        run = True
        clock = pygame.time.Clock()
        while run:
            try:
                clock.tick(self.FPS)
                if self.start_life:
                    self.evolve_universe()
                if sum(list(pygame.key.get_pressed())) != 0:
                    self.move_camera(pygame.key.get_pressed()) 
                for event in pygame.event.get():
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        self.select_cell(event.pos)
                    if event.type == pygame.QUIT:
                        run = False
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            run = False
                        if event.key == pygame.K_SPACE:
                            #print("Pressed space")
                            self.start_life = not self.start_life
                            if self.start_life:
                                self.store_grid()
                        if event.key == pygame.K_BACKSPACE:
                            if self.start_life:
                                self.start_life = not self.start_life
                            self.restore_grid()
                            self.render()
                pygame.display.update()
            except:
                pygame.quit()
                exit()
        pygame.quit()

In [3]:
print(cirq.__version__)
print(qiskit.__version__)
print(tfq.__version__)
print(pygame.__version__)

0.8.0
0.16.1
0.3.0
2.0.1
